In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:

off_train = pd.read_csv('Data/ccf_offline_stage1_train.csv')

In [5]:
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']

In [7]:
off_test = pd.read_csv('Data/ccf_offline_stage1_test_revised.csv')
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

In [8]:
on_train = pd.read_csv('Data/ccf_online_stage1_train.csv')
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

In [10]:
# 划分数据集
dataset3 = off_test
feature3 = off_train[((off_train.date>='20160315')&(off_train.date<='20160630'))|((off_train.date=='null')&(off_train.date_received>='20160315')&(off_train.date_received<='20160630'))]

In [11]:
dataset2 = off_train[(off_train.date_received>='20160515')&(off_train.date_received<='20160615')]
feature2 = off_train[(off_train.date>='20160201')&(off_train.date<='20160514')|((off_train.date=='null')&(off_train.date_received>='20160201')&(off_train.date_received<='20160514'))]

In [12]:
dataset1 = off_train[(off_train.date_received>='20160414')&(off_train.date_received<='20160514')]
feature1 = off_train[(off_train.date>='20160101')&(off_train.date<='20160413')|((off_train.date=='null')&(off_train.date_received>='20160101')&(off_train.date_received<='20160413'))]

In [13]:
#for dataset3
t = dataset3[['user_id']]

In [14]:
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()

/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
t1 = dataset3[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()

/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
t2 = dataset3[['user_id','coupon_id','date_received']]
# 将日期换为字符串格式
t2.date_received = t2.date_received.astype('str')
# 按用户id和优惠卷id分组后，对领取日期进行合并，中间用‘：’连接
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
# 对合并的领取日期用：分割，计其分割的数量
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
# 筛选掉领取优惠卷少于2张的
t2 = t2[t2.receive_number>1]
# 分别求出最早领取时间和最晚
t2['max_date_received'] = t2.date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
# 合并数据
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]

/home/silence/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [17]:
t2.dtypes

user_id              int64
coupon_id            int64
max_date_received    int64
min_date_received    int64
dtype: object

In [18]:
t3 = dataset3[['user_id','coupon_id','date_received']]
t3 = pd.merge(t3, t2, on=['user_id','coupon_id'], how='left')

In [19]:
# 填充缺失值 方便后面计算
t3.max_date_received = t3.max_date_received.fillna(20160630)
t3.min_date_received = t3.min_date_received.fillna(20160801)

In [20]:
t3.max_date_received = t3.max_date_received.astype('int')
t3.min_date_received = t3.min_date_received.astype('int')

In [22]:
#t3.date_received = t3.date_received.astype('int')

In [21]:
# t3 = t3.apply(pd.to_numeric, args=('coerce',))
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received

In [22]:
t3[t3.this_month_user_receive_same_coupon_firstone >= 0]

,user_id,coupon_id,date_received,max_date_received,min_date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
79,5568010,8596,20160722,20160722,20160722,0,0
80,5568010,8596,20160722,20160722,20160722,0,0
94,3644167,11463,20160710,20160713,20160706,3,4
95,3644167,11463,20160706,20160713,20160706,7,0
96,3644167,11463,20160708,20160713,20160706,5,2
97,3644167,11463,20160711,20160713,20160706,2,5
98,3644167,11463,20160713,20160713,20160706,0,7
120,3704071,10110,20160716,20160716,20160707,0,9
121,3704071,10110,20160707,20160716,20160707,9,0
132,3093769,8306,20160728,20160728,20160728,0,0


In [23]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [24]:
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

In [25]:
t3[t3.this_month_user_receive_same_coupon_firstone != -1]

,user_id,coupon_id,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
79,5568010,8596,20160722,1,1
80,5568010,8596,20160722,1,1
94,3644167,11463,20160710,0,0
95,3644167,11463,20160706,0,1
96,3644167,11463,20160708,0,0
97,3644167,11463,20160711,0,0
98,3644167,11463,20160713,1,0
120,3704071,10110,20160716,1,0
121,3704071,10110,20160707,0,1
132,3093769,8306,20160728,1,1


In [26]:
t3.count()

user_id                                         113640
coupon_id                                       113640
date_received                                   113640
this_month_user_receive_same_coupon_lastone     113640
this_month_user_receive_same_coupon_firstone    113640
dtype: int64

In [27]:
t4 = dataset3[['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()

/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
t5 = dataset3[['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()

/home/silence/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
t6 = dataset3[['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)

/home/silence/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [30]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [31]:
t7 = dataset3[['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]

In [34]:
other_feature3 = pd.merge(t1,t,on='user_id')

In [37]:
other_feature3 = pd.merge(other_feature3,t3,on=['user_id','coupon_id'])

In [39]:
other_feature3 = pd.merge(other_feature3,t4,on=['user_id','date_received'])

In [41]:
other_feature3 = pd.merge(other_feature3,t5,on=['user_id','coupon_id','date_received'])

In [43]:
other_feature3 = pd.merge(other_feature3,t7,on=['user_id','coupon_id','date_received'])

In [44]:
other_feature3

,user_id,coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,209,825,1,2,20160721,-1,-1,2,1,-1,-1
1,209,7557,1,2,20160721,-1,-1,2,1,-1,-1
2,215,5488,1,1,20160703,-1,-1,1,1,-1,-1
3,316,3992,1,1,20160721,-1,-1,1,1,-1,-1
4,417,12465,1,1,20160712,-1,-1,1,1,-1,-1
5,432,10438,1,1,20160706,-1,-1,1,1,-1,-1
6,448,10927,4,5,20160710,1,0,1,1,3,-1
7,448,10927,4,5,20160707,0,0,1,1,1,3
8,448,10927,4,5,20160706,0,0,1,1,2,1
9,448,10927,4,5,20160704,0,1,1,1,-1,2


In [45]:
other_feature3.to_csv('Data/other_feature3.csv',index=None)

In [47]:
other_feature3.shape

(116204, 11)